# Starbucks Prediction
### 2019 July 8
`Group 2`

In [ ]:
# beginning off by importing our libraries, as per usual
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB   #import Gaussian Bayes modeling function
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB
from pandas import read_excel

%matplotlib inline